In [1]:
! pip install flashtext
! pip install torch
! pip install sentencepiece
! pip install transformers
! pip install sentence-transformers
! pip install sense2vec

In [3]:
! pip install transformers==4.8.1 sentencepiece==0.1.95 flashtext==2.7 sentence-transformers==2.2.2 sense2vec==2.0.0 textwrap3==0.9.2 flashtext
! pip install git+https://github.com/boudinfl/pke.git
! pip install -r requirements.txt
from flashtext import KeywordProcessor
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline, BartTokenizer, BartForConditionalGeneration
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import string
import pke
import traceback
import spacy
from sense2vec import Sense2Vec
import numpy as np
import sentencepiece
spacy_model = spacy.load('en_core_web_sm')

nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.0 MB/s eta 0:00:00
  Using cached sentencepiece-0.1.95.tar.gz (508 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sense2vec-2.0.0-py2.py3-none-any.whl (39 kB)
  Using cached textwrap3-0.9.2-py2.py3-none-any.whl (12 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torchvision-0.17.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 k

/Users/kaushalpatil/Development/Question Generation Paper/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kaushalpatil/Development/Question Generation Paper/venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/kaushalpatil/Development/Question Generation Paper/venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kaushalpatil/nltk_data...
[nltk_data]   Packa

In [3]:
# from similarity.normalized_levenshtein import NormalizedLevenshtein
# normalized_levenshtein = NormalizedLevenshtein()
from sentence_transformers import SentenceTransformer
sentence_transformer_model = SentenceTransformer('msmarco-distilbert-base-v3')
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def summarizer(text, high = 500, low = 250):
    text = text.strip().replace("\n"," ")
    text = "summarize: "+text
    input_tokens = summary_tokenizer.batch_encode_plus ([text], return_tensors='pt', max_length=1024, truncation=True)['input_ids']
    encoded_ids = summary_model.generate (input_tokens, num_beams=4, length_penalty=2.0, max_length=high, min_length=low, no_repeat_ngram_size=3)
    summary = summary_tokenizer.decode(encoded_ids. squeeze (), skip_special_tokens=True)
    print(summary)
    return summary

In [6]:
def get_nouns_multipartite(content):
    out=[]
    try:
        extractor = pke.unsupervised.MultipartiteRank()
        extractor.load_document(input=content,language='en', spacy_model = spacy_model)
        pos = {'PROPN','NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(pos=pos)
        extractor.candidate_weighting(alpha=1.1,threshold=0.75,method='average')
        keyphrases = extractor.get_n_best(n=15)
        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()
    print(out)
    return out

In [7]:
def get_keywords(originaltext, summarytext):
    keywords = get_nouns_multipartite(originaltext)
    print ("keywords unsummarized: ",keywords)
    keyword_processor = KeywordProcessor()
    for keyword in keywords:
        keyword_processor.add_keyword(keyword)

    keywords_found = keyword_processor.extract_keywords(summarytext)
    keywords_found = list(set(keywords_found))
    print ("keywords_found in summarized: ",keywords_found)

    important_keywords =[]
    for keyword in keywords:
        if keyword in keywords_found:
            important_keywords.append(keyword)
    return important_keywords

# imp_keywords = get_keywords(text, summarized_text)
# print(imp_keywords)

In [8]:
def get_question(context, answer, model, tokenizer):
    text = "context: {} answer: {}".format(context, answer)
    encoding = tokenizer.encode_plus(
        text, max_length=384, pad_to_max_length=False, truncation=True, return_tensors="pt"
    ).to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    outs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        early_stopping=True,
        num_beams=5,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        max_length=72,
    )
    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    Question = dec[0].replace("question:", "")
    Question = Question.strip()
    return Question

In [1]:
text = '''Operating System lies in the category of system software. 
It basically manages all the resources of the computer. 
An operating system acts as an interface between the software and different parts of the computer or the computer hardware. 
The operating system is designed in such a way that it can manage the overall resources and operations of the computer. 
Operating System is a fully integrated set of specialized programs that handle all the operations of the computer. 
It controls and monitors the execution of all other programs that reside in the computer, which also includes application programs and other system software of the computer. 
Examples of Operating Systems are Windows, Linux, Mac OS, etc. 
An Operating System (OS) is a collection of software that manages computer hardware resources and provides common services for computer programs. 
The operating system is the most important type of system software in a computer system. 
The operating system helps in improving the computer software as well as hardware. 
Without OS, it became very difficult for any application to be user-friendly. 
The Operating System provides a user with an interface that makes any application attractive and user-friendly. 
The operating System comes with a large number of device drivers that make OS services reachable to the hardware environment. 
Each and every application present in the system requires the Operating System. 
The operating system works as a communication channel between system hardware and system software. 
The operating system helps an application with the hardware part without knowing about the actual hardware configuration. 
It is one of the most important parts of the system and hence it is present in every device, whether large or small device. 
Functions of the Operating System: Resource Management: The operating system manages and allocates memory, CPU time, and other hardware resources among the various programs and processes running on the computer. 
Process Management: The operating system is responsible for starting, stopping, and managing processes and programs. It also controls the scheduling of processes and allocates resources to them. 
Memory Management: The operating system manages the computer's primary memory and provides mechanisms for optimizing memory usage. 
Security: The operating system provides a secure environment for the user, applications, and data by implementing security policies and mechanisms such as access controls and encryption. 
Job Accounting: It keeps track of time and resources used by various jobs or users. 
File Management: The operating system is responsible for organizing and managing the file system, including the creation, deletion, and manipulation of files and directories. 
Device Management: The operating system manages input/output devices such as printers, keyboards, mice, and displays. It provides the necessary drivers and interfaces to enable communication between the devices and the computer. 
Networking: The operating system provides networking capabilities such as establishing and managing network connections, handling network protocols, and sharing resources such as printers and files over a network. 
User Interface: The operating system provides a user interface that enables users to interact with the computer system. This can be a Graphical User Interface (GUI), a Command-Line Interface (CLI), or a combination of both. 
Backup and Recovery: The operating system provides mechanisms for backing up data and recovering it in case of system failures, errors, or disasters. 
Virtualization: The operating system provides virtualization capabilities that allow multiple operating systems or applications to run on a single physical machine. This can enable efficient use of resources and flexibility in managing workloads. 
Performance Monitoring: The operating system provides tools for monitoring and optimizing system performance, including identifying bottlenecks, optimizing resource usage, and analyzing system logs and metrics. 
Time-Sharing: The operating system enables multiple users to share a computer system and its resources simultaneously by providing time-sharing mechanisms that allocate resources fairly and efficiently. 
System Calls: The operating system provides a set of system calls that enable applications to interact with the operating system and access its resources. System calls provide a standardized interface between applications and the operating system, enabling portability and compatibility across different hardware and software platforms. 
Error-detecting Aids: These contain methods that include the production of dumps, traces, error messages, and other debugging and error-detecting methods.'''

In [2]:
def generate_mcqs(text):
    summarized_text = summarizer(text)
    imp_keywords = get_keywords(text, summarized_text)
    id = 0
    questions = []
    for answer in imp_keywords:
        ques = get_question(summarized_text, answer, question_model, question_tokenizer)
        id = len(questions) + 1
        question_data = {
            "ID": id,  
            "Question": ques,
            "answer": answer.capitalize(),
        }
        questions.append(question_data)
    return questions

ans = generate_mcqs(text)

NameError: name 'summarizer' is not defined

In [25]:
import json
with open('result-0.json', 'w') as fp:
    json.dump(ans, fp)

In [26]:
with open('result-0.json', 'r') as f:
    data = json.load(f)
print(json.dumps(data, indent= 4))

[
    {
        "ID": 1,
        "Question": "What is one of the most important parts of a computer?",
        "answer": "Operating system"
    },
    {
        "ID": 2,
        "Question": "The operating system is one of the most important parts of what?",
        "answer": "System"
    },
    {
        "ID": 3,
        "Question": "An operating system is designed to manage the overall resources and operations of what?",
        "answer": "Computer"
    },
    {
        "ID": 4,
        "Question": "An operating system can enable efficient use of what?",
        "answer": "Resources"
    },
    {
        "ID": 5,
        "Question": "What does the operating system communicate with?",
        "answer": "System software"
    },
    {
        "ID": 6,
        "Question": "An operating system provides a secure environment for who?",
        "answer": "User"
    }
]


### Distractor Generation

In [ ]:
def filter_same_sense_words(original,wordlist):
    filtered_words=[]
    base_sense =original.split('|')[1]
    # print (base_sense)
    for eachword in wordlist:
        if eachword[0].split('|')[1] == base_sense:
            filtered_words.append(eachword[0].split('|')[0].replace("_", " ").title().strip())
    return filtered_words

def get_highest_similarity_score(wordlist,wrd):
    score=[]
    for each in wordlist:
        score.append(normalized_levenshtein.similarity(each.lower(),wrd.lower()))
    return max(score)

def sense2vec_get_words(word,s2v,topn,question):
    output = []
    # print ("word ",word)
    try:
        sense = s2v.get_best_sense(word, senses= ["NOUN", "PERSON","PRODUCT","LOC","ORG","EVENT","NORP","WORK OF ART","FAC","GPE","NUM","FACILITY"])
        most_similar = s2v.most_similar(sense, n=topn)
        output = filter_same_sense_words(sense,most_similar)
        # print ("Similar ",output)
    except:
        output =[]

    threshold = 0.6
    final=[word]
    checklist =question.split()
    for x in output:
        if get_highest_similarity_score(final,x)<threshold and x not in final and x not in checklist:
            final.append(x)

    return final[1:]

# def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
#     word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
#     word_similarity = cosine_similarity(word_embeddings)
#     keywords_idx = [np.argmax(word_doc_similarity)]
#     candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
#     for _ in range(top_n - 1):
#         candidate_similarities = word_doc_similarity[candidates_idx, :]
#         target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)
#         mmr = (lambda_param) * candidate_similarities - (1-lambda_param) * target_similarities.reshape(-1, 1)
#         mmr_idx = candidates_idx[np.argmax(mmr)]
#         keywords_idx.append(mmr_idx)
#         candidates_idx.remove(mmr_idx)
#     return [words[idx] for idx in keywords_idx]

def get_distractors(word, origsentence, sense2vecmodel, sentencemodel, top_n, lambdaval):
    distractors = sense2vec_get_words(word,sense2vecmodel,top_n,origsentence)
    # print ("distractors ",distractors)
    # if len(distractors) ==0:
    #     return distractors
    # distractors_new = [word.capitalize()]
    # distractors_new.extend(distractors)
    # embedding_sentence = origsentence+ " "+word.capitalize()
    # keyword_embedding = sentencemodel.encode([embedding_sentence])
    # distractor_embeddings = sentencemodel.encode(distractors_new)
    # max_keywords = min(len(distractors_new),5)
    # filtered_keywords = mmr(keyword_embedding, distractor_embeddings,distractors_new,max_keywords,lambdaval)
    # # filtered_keywords = filtered_keywords[1:]
    # final = [word.capitalize()]
    # for wrd in filtered_keywords:
    #     if wrd.lower() != word.lower():
    #         final.append(wrd.capitalize())
    # final = final[1:]
    # top1, top2, top3 = "", "", ""
    # sim1, sim2, sim3 = 0, 0, 0
    ans = []
    for i in distractors:
        ans.append([i, cosine_similarity(sentencemodel.encode([word]), sentencemodel.encode([i]))])
    ans.sort(key=lambda x: x[1], reverse=True)
    for i in range(len(ans)):
        ans[i] = ans[i][0]
    return ans[:3]